In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# import janitor #https://pyjanitor-devs.github.io/pyjanitor/

### Set Global Variables and Create Dataframes

Dataframe Name | Table Name
| --- | --- |
df_epp | Expenditures Per Pupil
df_nxp | Inexperienced Teachers and Principals
df_ocert | Teachers Teaching Out of Certification
df_y | Annual Regents Exams

In [2]:
df_epp = pd.read_excel("../Data/SRC2022/Expenditures Per Pupil.xlsx")
df_nxp = pd.read_excel("../Data/SRC2022/Inexperienced Teachers and Principals.xlsx")
df_ocert = pd.read_excel("../Data/SRC2022/Teachers Teaching Out of Certification.xlsx")
df_y = pd.read_excel("../Data/SRC2022/Annual Regents Exams.xlsx")

Variable Name | Meaning
| --- | --- |
chosen_test_subjects | The Annual Regents Exams we plan to use to measure school performance

In [3]:
chosen_test_subjects = ('Regents Common Core Algebra I', 'Regents Common Core English Language Art', 'Regents Phy Set/Earth Sci')

### Trimming Dataframes
#### Defining a Trimming Function that will only keep the columns we wish and only keep the rows that have our desired YEAR and SUBGROUP_NAME of "All Students"

In [4]:
def trimmer(df: pd.DataFrame, *column_names, year: int=2021, year_col_name: str="YEAR")->pd.DataFrame:
    """
    Keeps only the columns specified, only the rows whose year agrees with the year argument, 
    and if there is a SUBGROUP column then it keeps only rows with the subgroup classification 
    of "All Students". 
    This is NOT an in-place method. 
    """
    # Trimming Rows
    if year_col_name in df.columns:
        df = df.loc[df[year_col_name]==year]
    if "SUBGROUP_NAME" in df.columns:
        df = df.loc[df["SUBGROUP_NAME"]=="All Students"] #<-----This drops all rows that do not have "All Students" as the "SUBGROUP" column entry. 
    if "SUBJECT" in df.columns:
        df = df.loc[df.SUBJECT.isin(chosen_test_subjects)]
    # Trimming Columns
    return df[list(column_names)]

Note that we no longer have to look for ENTITY_CD ENTRIES that don't end in 0000.

#### Applying the Trimming Function

In [5]:
# Expenditures Per Pupil: Keep only rows with the year 2021 and keeping only school ID, Name and Normalized Expenditures
df_epp_t = trimmer(df_epp,'ENTITY_NAME', 'ENTITY_CD', 'PER_FED_STATE_LOCAL_EXP')
df_nxp_t = trimmer(df_nxp,'ENTITY_CD', 'PER_TEACH_INEXP')
df_ocert_t = trimmer(df_ocert,'ENTITY_CD', "PER_OUT_CERT")

# df_y: Drop NaN values and make INSTITUTION_ID a string
df_y_c = df_y.dropna(subset=['INSTITUTION_ID'], ignore_index=True)

# df_y: Trim and Pivot
df_y_t = trimmer(df_y_c, 'INSTITUTION_ID', 'ENTITY_CD', 'ENTITY_NAME', 'SUBJECT', "PER_PROF")

We had a lot of trouble with pivoting before. Now, hopefully the code all works well and we can just pivot. 
#### Checking for duplicates. 

In [6]:
df_y_t.ENTITY_CD.value_counts()

ENTITY_CD
211103040000    3
261600860985    3
220909040010    3
211701040001    3
211103040001    3
               ..
310200010177    1
310200010167    1
310200010131    1
310200010126    1
671002040001    1
Name: count, Length: 429, dtype: int64

Success! We have at most 3 instances for each ENTITY_CD, one for each type of exam we decided on. 

But if we look on the row of the last cell, it says "Length: <font color='red'><del>218</del></font> 429". So there are only <font color='red'><del>218</del></font> 429 values. That means there are only <font color='red'><del>218</del></font> 429 schools in the list now. We get the same number when using the INSTITUTION_ID column, so it's not the choice of id:

In [28]:
df_y_t.INSTITUTION_ID.nunique()

429

But if we go back to the original Regents Exams Dataframe, we get almost 3,000 schools. Does this mean that 90% of these didn't have an exam entry for our matching tests? 

In [8]:
df_y.ENTITY_CD.value_counts()

ENTITY_CD
60601040000     214
60601040003     212
62901040000     195
62901040002     195
10100860907     193
               ... 
320900010323      1
320900010328      1
261600010045      1
331700011524      1
342400010113      1
Name: count, Length: 2974, dtype: int64

#### Pivoting the exam results

In [60]:
# It turns out that using lists with single elements was what was keeping the data in the weird form. I had tried .reset_index() a few times and it never worked. This was because I was using the syntax from .pivot_table() instead of .pivot(). Now it works! 
df_y_p = df_y_t.pivot(index='ENTITY_CD', 
                            values="PER_PROF", 
                            columns="SUBJECT"
                            ).reset_index()
df_y_p.columns.name = None #Gets rid of the "SUBJECT" name for the indices https://www.youtube.com/watch?v=7flWNolPhsc
df_y_p.head()

,ENTITY_CD,Regents Common Core Algebra I,Regents Common Core English Language Art,Regents Phy Set/Earth Sci
0,10100860907,40,70,00
1,10615020000,100,NaN,100
2,10615020001,100,NaN,100
3,10623060006,s,NaN,NaN
4,10623060007,s,NaN,NaN


And we can see that the table pivots with no trouble, yay! 

In [10]:
df_y_p.head()

ENTITY_CD                      PER_PROF  \
SUBJECT              Regents Common Core Algebra I   
0        10100860907                            40   
1        10615020000                           100   
2        10615020001                           100   
3        10623060006                             s   
4        10623060007                             s   

                                                                            
SUBJECT Regents Common Core English Language Art Regents Phy Set/Earth Sci  
0                                             70                        00  
1                                            NaN                       100  
2                                            NaN                       100  
3                                            NaN                       NaN  
4                                            NaN                       NaN

### Merging the Tables
Rather than having 3 different merge functions, we can use the reduce function from the functools library and define one merging function that will merge each cumulative pair of dataframes. 

In [65]:
from functools import reduce

# Data Frames List
data_frames_to_merge = [df_epp_t, df_nxp_t, df_ocert_t, df_y_p]

# Merging Function
def merge_function(left_df, right_df):
    return pd.merge(left_df, right_df, on='ENTITY_CD',
                                            how='inner'
                                            )

# Merge 'em! 
df = reduce(merge_function, data_frames_to_merge)

<del>The merge failed because the pivot table has rows/columns at the top that are wonky. We need to look into how to fix that.</del>
The merge was successful! 

In [66]:
df

,ENTITY_NAME,ENTITY_CD,PER_FED_STATE_LOCAL_EXP,PER_TEACH_INEXP,PER_OUT_CERT,Regents Common Core Algebra I,Regents Common Core English Language Art,Regents Phy Set/Earth Sci
0,PINE VALLEY CSD (SOUTH DAYTON),60601040000,24772.0,21.0,4.0,63,93,NaN
1,PINE VALLEY CENTRAL JR-SR HIGH SCH,60601040003,16169.0,22.0,7.0,64,93,NaN
2,GREEN TECH HIGH CHARTER SCHOOL,10100860907,13917.0,8.0,61.0,40,70,00
3,MENANDS UFSD,10615020000,18122.0,7.0,3.0,100,NaN,100
4,MENANDS SCHOOL,10615020001,12333.0,7.0,3.0,100,NaN,100
...,...,...,...,...,...,...,...,...
424,MATTITUCK-CUTCHOGUE UFSD,581012020000,NaN,12.0,0.0,NaN,NaN,79
425,MATTITUCK JUNIOR-SENIOR HIGH SCHOOL,581012020001,23810.0,13.0,0.0,NaN,NaN,79
426,SPENCER-VAN ETTEN MIDDLE SCHOOL,600801040001,15171.0,22.0,7.0,100,NaN,NaN
427,GROTON CSD,610501040000,20384.0,32.0,5.0,NaN,72,NaN


It now looks like we could turn our attention to the low numbers in the exam columns and find a way to address the "s" characters in them that is preventing them from being numeric. 

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429 entries, 0 to 428
Data columns (total 8 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   ENTITY_NAME                               429 non-null    object 
 1   ENTITY_CD                                 429 non-null    int64  
 2   PER_FED_STATE_LOCAL_EXP                   425 non-null    float64
 3   PER_TEACH_INEXP                           427 non-null    float64
 4   PER_OUT_CERT                              425 non-null    float64
 5   Regents Common Core Algebra I             341 non-null    object 
 6   Regents Common Core English Language Art  106 non-null    object 
 7   Regents Phy Set/Earth Sci                 121 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 26.9+ KB
